In [1]:
import tensorflow as tf
import numpy as np
from keras  import models, layers
import keras
from PIL import Image, ImageDraw, ImageFont
from sklearn.model_selection import train_test_split
import random
import cv2
import os

2024-01-29 17:12:01.987744: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-29 17:12:02.013265: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-29 17:12:02.013285: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-29 17:12:02.014073: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-29 17:12:02.018701: I tensorflow/core/platform/cpu_feature_guar

In [2]:
width = 30
height = 40
images = []
labels = []
letters = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']


In [3]:
def letter_to_image(letter: str, start: tuple[float, float], rotate, flag: bool, font: str, font_size: int):
    img = Image.new('L', (width, height), color='Black')
    #font_size used to be fixed-45 
    font = ImageFont.truetype(font, size=font_size)
    img_draw = ImageDraw.Draw(img)
    img_draw.text(text=letter, fill=255, font=font, xy = start)

    img = img.rotate(rotate) 
    if flag:
        draw = ImageDraw.Draw(img) 
        draw.line((random.randint(0, width - 1), random.randint(0, height - 1),
                   random.randint(0, width - 1), random.randint(0, height - 1)),
                  fill=255, width=2)
    
    return img

In [4]:
fonts = os.listdir('fonts')
fonts =  ["fonts/" + font for font in fonts]
print(fonts)

['fonts/Roboto-Bold.ttf', 'fonts/times new roman bold.ttf', 'fonts/Raleway-Bold.ttf', 'fonts/G_ari_bd.TTF', 'fonts/SLC_.ttf']


In [5]:
for font in fonts:
    for size in range(42, 48, 2):
        for l in letters:
            for x in range(-4, 6, 2):
                for y in range(-10, 5, 2):
                    for r in range(-30, 30, 2):
                        images.append(letter_to_image(l, (x, y), r, random.randint(1, 8) == 1, font, size))
                        labels.append(ord(l) - ord('A'))

# for i in range(0, 300):
#     images[i].save('data/result{i}.png'.format(i = i))

# letter_to_image('T', (1, -0), 30, False).save("result.png")

In [6]:
# old_image = images
# old_labels = labels

In [7]:
num_classes = len(letters)
# images = np.array(old_image) / 255.0
# labels = np.array(old_labels)
images = np.array(images) / 255.0
labels = np.array(labels)

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.1)
train_labels = keras.utils.to_categorical(train_labels, num_classes=num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes=num_classes)
print(images.shape)
print(train_labels)

(468000, 40, 30)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [8]:
num_classes = len(letters)
# Define the CNN architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Adjust the number of neurons in the output layer
])

# Compile the model with appropriate loss and metrics for multi-class classification
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Categorical Crossentropy for multi-class classification
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=4, validation_data=(test_images, test_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)


2024-01-29 17:14:04.513356: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-29 17:14:04.963035: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-29 17:14:04.963252: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/4


2024-01-29 17:15:41.296942: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-29 17:15:45.010460: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-29 17:15:55.682468: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-29 17:15:55.813879: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f3108bf8300 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-29 17:15:55.813919: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2024-01-29 17:15:55.870497: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1706535956.136291   14845 devic

13163/13163 [==============================] - 112s 6ms/step - loss: 0.1777 - accuracy: 0.9389 - val_loss: 0.0588 - val_accuracy: 0.9757
Epoch 2/4
13163/13163 [==============================] - 43s 3ms/step - loss: 0.0484 - accuracy: 0.9793 - val_loss: 0.0470 - val_accuracy: 0.9803
Epoch 3/4
13163/13163 [==============================] - 40s 3ms/step - loss: 0.0363 - accuracy: 0.9839 - val_loss: 0.0322 - val_accuracy: 0.9852
Epoch 4/4
13163/13163 [==============================] - 38s 3ms/step - loss: 0.0306 - accuracy: 0.9860 - val_loss: 0.0396 - val_accuracy: 0.9833
1463/1463 - 2s - loss: 0.0396 - accuracy: 0.9833 - 2s/epoch - 1ms/step

Test accuracy: 0.9832906126976013


In [9]:
model.save('my_model.keras')

In [10]:
load = models.load_model('my_model.keras')


In [11]:
img = cv2.imread("letter_6.jpg", cv2.IMREAD_GRAYSCALE)
print(img.shape)
print(load.predict(np.array([img])))

(40, 30)
1/1 [==============================] - 0s 192ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1.]]
